## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-22-20-48-49 +0000,nyt,Pickett Fire Forces Evacuations in Napa County,https://www.nytimes.com/2025/08/21/us/pickett-...
1,2025-08-22-20-44-45 +0000,nypost,Stray dog slaughters 23 baby sea turtles just ...,https://nypost.com/2025/08/22/us-news/stray-do...
2,2025-08-22-20-42-08 +0000,nyt,F.B.I. Searches Home and Office of John Bolton...,https://www.nytimes.com/2025/08/22/us/politics...
3,2025-08-22-20-40-28 +0000,nypost,LI pol rips MTA’s fare hike ‘cash grab’ after ...,https://nypost.com/2025/08/22/us-news/li-pol-r...
4,2025-08-22-20-36-10 +0000,nypost,Broadway theater owners fight back in iconic P...,https://nypost.com/2025/08/22/us-news/does-not...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,63
271,new,19
48,will,16
290,up,14
139,national,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
90,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...,210
140,2025-08-22-16-19-00 +0000,wsj,Defense Secretary Pete Hegseth has authorized ...,https://www.wsj.com/politics/policy/defense-se...,136
103,2025-08-22-17-46-20 +0000,nypost,How John Bolton’s controversial 2020 memoir sp...,https://nypost.com/2025/08/22/us-news/how-john...,129
330,2025-08-21-22-11-40 +0000,latimes,California voters will decide redistricting in...,https://www.latimes.com/california/story/2025-...,117
112,2025-08-22-17-22-48 +0000,nyt,"Who Is John Bolton, Trump’s Former National Se...",https://www.nytimes.com/2025/08/22/us/politics...,117


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
90,210,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...
78,77,2025-08-22-18-33-00 +0000,wsj,Canada’s decision to drop tariffs on most U.S....,https://www.wsj.com/articles/canadas-rollback-...
311,62,2025-08-21-23-43-37 +0000,nypost,California Gov. Gavin Newsom signs redistricti...,https://nypost.com/2025/08/21/us-news/californ...
262,61,2025-08-22-04-36-24 +0000,wapo,"Erik Menendez, who was convicted of killing hi...",https://www.washingtonpost.com/nation/2025/08/...
6,56,2025-08-22-20-32-55 +0000,nyt,Jerome Powell Suggests Fed Will Soon Cut Inter...,https://www.nytimes.com/2025/08/22/business/po...
216,55,2025-08-22-10-08-00 +0000,bbc,India top court shelves plan to lock up Delhi'...,https://www.bbc.com/news/articles/c5yejnze4p1o...
164,52,2025-08-22-14-55-00 +0000,wsj,A federal judge ordered the government to dism...,https://www.wsj.com/politics/policy/federal-ju...
317,40,2025-08-21-23-02-09 +0000,nypost,Israeli stats show civilian death rate in Gaza...,https://nypost.com/2025/08/21/world-news/israe...
201,39,2025-08-22-11-36-52 +0000,nypost,Ukraine obliterates critical part of Russia’s ...,https://nypost.com/2025/08/22/world-news/ukrai...
196,38,2025-08-22-12-05-57 +0000,wapo,He agreed to conduct a band when he turned 100...,https://www.washingtonpost.com/lifestyle/2025/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
